# Tensorflow Wide and Deep Tutorial
This notebook demos how to run tensorflow [wide](https://www.tensorflow.org/tutorials/wide)
and [wide and deep](https://www.tensorflow.org/tutorials/wide_and_deep) tutorial 

In [1]:
# As usural, we open a database connection.   
import dg.conn
import dg.xtable

con = dg.conn.Conn(user="ftian")
print (con.ver)

4


In [4]:
# Step 1: load data  Only need to run once.
from dg.dsutil.csv import CsvXt

In [3]:
# Load data.  For all continous feature, we use float.   Categorical, text.
train = CsvXt('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
train.add_col('age', 'float') 
train.add_col('workclass', 'text') 
train.add_col('fnlwgt', 'float')
train.add_col('education', 'text')
train.add_col('education_num', 'float')
train.add_col('marital_status', 'text')
train.add_col('occupation', 'text')
train.add_col('relationship', 'text')
train.add_col('race', 'text')
train.add_col('gender', 'text')
train.add_col('capital_gain', 'float')
train.add_col('capital_loss', 'float')
train.add_col('hours_per_week', 'float')
train.add_col('native_country', 'text')
train.add_col('income', 'text')
xtrain = train.xtable(con)
# print (xtrain.select(samplerows=10).show())

In [4]:
# You only need to run ctas once.
xtrain.ctas("widedeep_train")

In [5]:
# Load data.  For all continous feature, we use float.   Categorical, text.
# copypaste from above.
test = CsvXt('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test')
test.add_col('age', 'float') 
test.add_col('workclass', 'text') 
test.add_col('fnlwgt', 'float')
test.add_col('education', 'text')
test.add_col('education_num', 'float')
test.add_col('marital_status', 'text')
test.add_col('occupation', 'text')
test.add_col('relationship', 'text')
test.add_col('race', 'text')
test.add_col('gender', 'text')
test.add_col('capital_gain', 'float')
test.add_col('capital_loss', 'float')
test.add_col('hours_per_week', 'float')
test.add_col('native_country', 'text')
test.add_col('income', 'text')
xtest = test.xtable(con)
# # print (xtrain.select(samplerows=10).show())

In [6]:
# Fix: raw data set test file has a trailing dot.
xtest = xtest.select(select='''
    age, workclass, fnlwgt, education, education_num, 
    marital_status, occupation, relationship, race, gender, 
    capital_gain, capital_loss, hours_per_week, native_country,
    case when income = '>50K.' then '>50K'
         when income = '<=50K.' then '<=50K'
    end as income
''')

In [9]:
xtest.ctas('widedeep_test')

In [2]:

print (dg.xtable.fromQuery(con, """select 'num_train', count(*) from widedeep_train 
                                   union all
                                   select 'num_test', count(*) from widedeep_test
                                   """).show())
# print (dg.xtable.fromTable(con, "widedeep_train").select(samplerows=10).show())
# print (dg.xtable.fromTable(con, "widedeep_test").select(samplerows=10).show())

+------------+---------+
| ?column?   |   count |
|------------+---------|
| num_train  |   32561 |
| num_test   |   16281 |
+------------+---------+


In [8]:
print(xtest.select(samplerows=10).show())

+-------+-------------+----------+--------------+-----------------+--------------------+-------------------+----------------+--------+----------+----------------+----------------+------------------+------------------+----------+
|   age | workclass   |   fnlwgt | education    |   education_num | marital_status     | occupation        | relationship   | race   | gender   |   capital_gain |   capital_loss |   hours_per_week | native_country   | income   |
|-------+-------------+----------+--------------+-----------------+--------------------+-------------------+----------------+--------+----------+----------------+----------------+------------------+------------------+----------|
|    44 | Local-gov   |   282069 | Assoc-voc    |              11 | Married-civ-spouse | Farming-fishing   | Husband        | White  | Male     |              0 |              0 |               40 | United-States    | <=50K    |
|    36 | Private     |   124111 | HS-grad      |               9 | Divorced        

In [ ]:
localtab = dg.xtable.fromTable(con, 'iris')
print(localtab.show(tablefmt='fancy_grid'))